# LLM Arena-as-a-Judge
In this tutorial, we will explore how to implement the LLM Arena-as-a-Judge approach to evaluate large language model outputs. Instead of assigning isolated numerical scores to each response, this method performs head-to-head comparisons between outputs to determine which one is better — based on criteria you define, such as helpfulness, clarity, or tone.

We'll use OpenAI's `GPT-4.1` and `Gemini 2.5 Pro` to generate responses, and leverage `GPT-5` as the judge to evaluate their outputs. For demonstration, we’ll work with a simple email support scenario, where the context is as follows:

```
Dear Support,  
I ordered a wireless mouse last week, but I received a keyboard instead.  
Can you please resolve this as soon as possible?  
Thank you,  
John  
```



## Installing the dependencies

In [23]:
!pip install deepeval google-genai openai

In this tutorial, you’ll need API keys from both OpenAI and Google.

* Google API Key: Visit https://aistudio.google.com/apikey
 to generate your key.

* OpenAI API Key: Go to https://platform.openai.com/settings/organization/api-keys
 and create a new key. If you’re a new user, you may need to add billing information and make a minimum payment of $5 to activate API access.

Since we’re using Deepeval for evaluation, the OpenAI API key is required

In [82]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass('Enter OpenAI API Key: ')
os.environ['GOOGLE_API_KEY'] = getpass('Enter Google API Key: ')

Enter OpenAI API Key: ··········
Enter Google API Key: ··········


## Defining the context
Next, we’ll define the context for our test case. In this example, we’re working with a customer support scenario where a user reports receiving the wrong product. We’ll create a context_email containing the original message from the customer and then build a prompt to generate a response based on that context.

In [72]:
from deepeval.test_case import ArenaTestCase, LLMTestCase, LLMTestCaseParams
from deepeval.metrics import ArenaGEval

context_email = """
Dear Support,
I ordered a wireless mouse last week, but I received a keyboard instead.
Can you please resolve this as soon as possible?
Thank you,
John
"""

prompt = f"""
{context_email}
--------

Q: Write a response to the customer email above.
"""

## OpenAI Model Response

In [58]:
from openai import OpenAI
client = OpenAI()

def get_openai_response(prompt: str, model: str = "gpt-4.1") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [60]:
openAI_response = get_openai_response(prompt=prompt)

## Gemini Model Response

In [62]:
from google import genai
client = genai.Client()

def get_gemini_response(prompt, model="gemini-2.5-pro"):
    response = client.models.generate_content(
        model=model,
        contents=prompt
    )
    return response.text

In [67]:
geminiResponse = get_gemini_response(prompt=prompt)

## Defining the Arena Test Case
Here, we set up the ArenaTestCase to compare the outputs of two models — GPT-4 and Gemini — for the same input prompt. Both models receive the same context_email, and their generated responses are stored in openAI_response and geminiResponse for evaluation.

In [77]:
a_test_case = ArenaTestCase(
    contestants={
        "GPT-4": LLMTestCase(
            input="Write a response to the customer email above.",
            context=[context_email],
            actual_output=openAI_response,
        ),
        "Gemini": LLMTestCase(
            input="Write a response to the customer email above.",
            context=[context_email],
            actual_output=geminiResponse,
        ),
    },
)

## Setting Up the Evaluation Metric
Here, we define the ArenaGEval metric named `Support Email Quality`. The evaluation focuses on empathy, professionalism, and clarity — aiming to identify the response that is understanding, polite, and concise. The evaluation considers the context, input, and model outputs, using GPT-5 as the evaluator with verbose logging enabled for better insights.

In [78]:
metric = ArenaGEval(
    name="Support Email Quality",
    criteria=(
        "Select the response that best balances empathy, professionalism, and clarity. "
        "It should sound understanding, polite, and be succinct."
    ),
    evaluation_params=[
        LLMTestCaseParams.CONTEXT,
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
    ],
    model="gpt-5",
    verbose_mode=True
)

## Running the Evaluation

In [79]:
metric.measure(a_test_case)

Output()

**************************************************

Support Email Quality [Arena GEval] Verbose Logs

**************************************************

Criteria:
Select the response that best balances empathy, professionalism, and clarity. It should sound understanding, 
polite, and be succinct. 
 
Evaluation Steps:
[
    "From the Context and Input, identify the user’s intent, needs, tone, and any constraints or specifics to be 
addressed.",
    "Verify the Actual Output directly responds to the Input, uses relevant details from the Context, and remains 
consistent with any constraints.",
    "Evaluate empathy: check whether the Actual Output acknowledges the user’s situation/feelings from the 
Context/Input in a polite, understanding way.",
    "Evaluate professionalism and clarity: ensure respectful, blame-free tone and concise, easy-to-understand 
wording; choose the response that best balances empathy, professionalism, and succinct clarity."
] 
 
Winner: GPT-4
 
Reason: GPT-4 delivers a single, concise, and professional email that directly addresses the context (acknowledges 
receiving a keyboard instead of the ordered wireless mouse), apologizes, and clearly outlines next steps (send the 
correct mouse and provide return instructions) with a polite verification step (requesting a photo). This best 
matches the request to write a response and balances empathy and clarity. In contrast, Gemini includes multiple 
options with meta commentary, which dilutes focus and fails to provide one clear reply; while empathetic and 
detailed (e.g., acknowledging frustration and offering prepaid labels), the multi-option format and an 
over-assertive claim of already locating the order reduce professionalism and succinct clarity compared to GPT-4.

======================================================================

'GPT-4'

The evaluation results show that GPT-4 outperformed the other model in generating a support email that balanced empathy, professionalism, and clarity. GPT-4’s response stood out because it was concise, polite, and action-oriented, directly addressing the situation by apologizing for the error, confirming the issue, and clearly explaining the next steps to resolve it, such as sending the correct item and providing return instructions. The tone was respectful and understanding, aligning perfectly with the user’s need for a clear and empathetic reply. In contrast, Gemini’s response, while empathetic and detailed, included multiple response options and unnecessary commentary, which reduced its clarity and professionalism. This result highlights GPT-4’s ability to deliver focused, customer-centric communication that feels both professional and considerate.

## Visualizing the responses in a table

In [81]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.DataFrame({
    "Model": ["Gemini", "OpenAI"],
    "Response": [geminiResponse, openAI_response]
})
df

,Model,Response
0,Gemini,"Of course. Here are a few options, ranging from standard and professional to more proactive.\n\n### Option 1: Standard & Professional (Most Common Choice)\n\nThis is a clear, polite, and effective response that solves the problem efficiently.\n\n**Subject: Re: Your Recent Order - Incorrect Item Received**\n\nDear [Customer Name],\n\nThank you for contacting us and for bringing this to our attention.\n\nI am very sorry to hear that you received a keyboard instead of the wireless mouse you ordered. I understand this is frustrating, and I sincerely apologize for our mistake.\n\nWe want to get the correct item to you as quickly as possible. To help me locate your order, could you please reply with your order number?\n\nOnce I have that, I will immediately ship the correct wireless mouse to you. I will also email you a pre-paid return label so you can send the keyboard back to us at no cost.\n\nWe appreciate your patience and look forward to resolving this for you.\n\nBest regards,\n\n[Your Name]\nCustomer Support Team\n\n---\n\n### Option 2: Proactive & Customer-Focused\n\nThis option goes a step further by shipping the new item immediately, showing extra trust in the customer and prioritizing speed.\n\n**Subject: Getting Your Correct Order Shipped Immediately**\n\nDear [Customer Name],\n\nThank you for your email. I am so sorry about the mix-up with your recent order. Receiving the wrong item is definitely not the experience we want for our customers, and we apologize for the inconvenience.\n\nI have already located your order based on your email address and have arranged for the correct wireless mouse to be shipped to you today via express shipping. You should receive a separate email with the new tracking information shortly.\n\nRegarding the keyboard you mistakenly received, there's no rush. I've attached a pre-paid return label to this email. Please feel free to use the original packaging to send it back to us at your convenience.\n\nAgain, we sincerely apologize for the error. If there is anything else I can help you with, please don't hesitate to ask.\n\nSincerely,\n\n[Your Name]\nCustomer Support Team\n\n---\n\n### Option 3: Concise & Friendly\n\nThis version is slightly less formal and gets straight to the point, which can be effective for some brands.\n\n**Subject: Re: Wrong item in your order**\n\nHi there,\n\nThanks for reaching out, and I'm so sorry we sent you a keyboard instead of a mouse! Let's get that fixed for you right away.\n\nCould you please reply with your order number?\n\nAs soon as I have it, I'll get the correct wireless mouse shipped out to you and send you a free return label for the keyboard.\n\nApologies again for the mix-up!\n\nAll the best,\n\n[Your Name]\nCustomer Support"
1,OpenAI,"Subject: Re: Incorrect Item Received – Order Support\n\nDear [Customer Name],\n\nThank you for reaching out to us, and I apologize for the mix-up with your order.\n\nWe’re sorry you received a keyboard instead of the wireless mouse you ordered. To resolve this as quickly as possible, could you please reply to this email with a photo of the item you received? Once we have this, we will arrange for the correct mouse to be sent to you right away and provide instructions for returning the incorrect item.\n\nThank you for your patience, and we look forward to resolving this for you.\n\nBest regards, \n[Your Name] \nCustomer Support Team \n[Company Name]"
